Plan de la tâche - Stakeholders' map

1. Création des types de données
2. Création des tables csv
3. Importation dans neo4j sous forme de noeuds-relations
4. Design des requêtes
5. Possibilité d'interroger la base via un notebook Jupyter


Lien du github : https://github.com/MaximeCapron/foodforneo4j

# 5. Possibilité d'interroger la base

In [ ]:
!pip install py2neo
# If you see warnings or errors when you run this command, re-run the command. It should run with no errors.

In [1]:
import numpy as np
import pandas as pd
from py2neo import Graph

In [2]:
graph = Graph("bolt://3.90.65.37:34875", auth=("neo4j", "volumes-capes-retrievals"))

## Initialiser le graph

In [ ]:
def initialize():
    
    code_initialisation = """

    // Supprimer tout ce qui préexiste pour avoir une feuille propre

    MATCH (n)
    DETACH DELETE n ;


    // Importer les noeuds 

    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Humains.csv' AS row FIELDTERMINATOR ';'
    MERGE (p:Person {id: row.index})
    ON CREATE SET p.nom = row.Nom, p.position_A = row.Position_A, p.position_B = row.Position_B, p.position_C = row.Position_C, p.position_D = row.Position_D, p.position_E = row.Position_E;


    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Entreprises.csv' AS row FIELDTERMINATOR ';'
    MERGE (c:Company {id: row.index})
    ON CREATE SET c.titre = row.Titre;


    // Importer les relations

    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Relations1.csv' AS rel FIELDTERMINATOR ';'
    MATCH (p1 {id: rel.index})
    MATCH (p2 {id: rel.index_relation})
    MERGE (p1)-[r:CONNAISSANCE]->(p2)
    ON CREATE SET r.contexte = rel.contexte;


    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Relations2.csv' AS rel FIELDTERMINATOR ';'
    MATCH (p1 {id: rel.index})
    MATCH (p2 {id: rel.index_relation})
    MERGE (p1)-[r:SUPERIEUR]->(p2) ;


    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Relations3.csv' AS rel FIELDTERMINATOR ';'
    MATCH (c1 {id: rel.index})
    MATCH (c2 {id: rel.index_relation})
    MERGE (c1)-[r:FILIALE]->(c2) ;


    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Relations4.csv' AS rel FIELDTERMINATOR ';'
    MATCH (p {id: rel.index})
    MATCH (c {id: rel.index_relation})
    MERGE (p)-[r:EMPLOYE_DANS]->(c)
    ON CREATE SET r.position = rel.position

    """
    
    graph.run(code_initialisation)

## Ajouter ou supprimer un noeud

2 solutions :
- si vous voulez faire quelque chose de lourd, l'option préférable est probablement de modifier les bases de données, en ajoutant, modifiant ou supprimant des lignes, puis en faisant tourner la cellule ci-dessus.
- si vous voulez simplement ajouter ou supprimer un noeud en vous basant sur ses caractéristiques, c'est faisable avec les fonctions ci-dessous.

Vous souhaitez ajouter un nouveau personnage. Il s'appelle Bob, on lui donne l'identifiant 5 (au hasard), et il est boulanger.

In [3]:
dic = {"id":5,"nom":"Bob","emploi":"Boulanger"}

In [4]:
def add_person(dic):
    code = "MERGE (p:Person {"
    for i in dic.keys():
        if type(dic[i]) == str:
            code += i + ":'" + dic[i] + "',"
        elif type(dic[i]) == int or type(dic[i]) == float:
            code += i + ":" + str(dic[i]) + ","
        else:
            raise "type inconnu"
    code = code[:-1] + "})"
    # return code
    graph.run(code)

In [5]:
add_person(dic)

Supprimons Bob.

In [6]:
char = {"nom":"Bob"}

In [9]:
def suppr_person(char):
    code = "MATCH (p:Person {"
    for i in char.keys():
        if type(char[i]) == str:
            code += i + ":'" + char[i] + "',"
        elif type(char[i]) == int or type(char[i]) == float:
            code += i + ":" + str(char[i]) + ","
        else:
            raise "type inconnu"
    code = code[:-1] + "}) DETACH DELETE p"
    graph.run(code)

In [10]:
suppr_person(char)

## Ajouter ou supprimer une relation

Supposons que l'on a deux noeuds, Bob et Joe (les créer si besoin), et que l'on veut créer une relation entre eux deux. Précisions une caractéristique pour chacun d'entre deux et la nature de la relation, et allons-y !

In [11]:
n1 = {"nom":"Bob"}
n2 = {"nom":"Joe"}
rel = "AMI"
dir_rel = 2

NB. dir_rel représente la pointe de la flèche de la relation. dir_rel peut donc être égal à 0, 1 ou 2 (en fonction de si la relation n'a pas de sens particulier, ou bien pointe vers n1, ou bien pointe vers n2).

In [12]:
def add_relation(n1,n2,rel,dir_rel):
    code = "MATCH (p1:Person {"
    for i in n1.keys():
        if type(n1[i]) == str:
            code += i + ":'" + n1[i] + "',"
        elif type(n1[i]) == int or type(n1[i]) == float:
            code += i + ":" + str(n1[i]) + ","
        else:
            raise "type inconnu"
    code = code[:-1] + "}) MATCH (p2:Person {"    
    for i in n2.keys():
        if type(n2[i]) == str:
            code += i + ":'" + n2[i] + "',"
        elif type(n2[i]) == int or type(n2[i]) == float:
            code += i + ":" + str(n2[i]) + ","
        else:
            raise "type inconnu"
    code = code[:-1] + "}) "
    if dir_rel == 0 :
        code += "MERGE (p1)-[r : "+rel+"]-(p2)"
    elif dir_rel == 1 :
        code += "MERGE (p1)<-[r : "+rel+"]-(p2)"
    elif dir_rel == 2 :
        code += "MERGE (p1)-[r : "+rel+"]->(p2)"
    else :
        raise "Wrong dir_rel"
        
    graph.run(code)

In [13]:
add_relation(n1,n2,rel,dir_rel)

Supprimons cette relation.

In [14]:
def suppr_relation(n1,n2,rel,dir_rel):
    code = "MATCH (p1:Person {"
    for i in n1.keys():
        if type(n1[i]) == str:
            code += i + ":'" + n1[i] + "',"
        elif type(n1[i]) == int or type(n1[i]) == float:
            code += i + ":" + str(n1[i]) + ","
        else:
            raise "type inconnu"
    code = code[:-1] + "}) MATCH (p2:Person {"    
    for i in n2.keys():
        if type(n2[i]) == str:
            code += i + ":'" + n2[i] + "',"
        elif type(n2[i]) == int or type(n2[i]) == float:
            code += i + ":" + str(n2[i]) + ","
        else:
            raise "type inconnu"
    code = code[:-1] + "}) "
    if dir_rel == 0 :
        code += "MATCH (p1)-[r : "+rel+"]-(p2) "
    elif dir_rel == 1 :
        code += "MATCH (p1)<-[r : "+rel+"]-(p2) "
    elif dir_rel == 2 :
        code += "MATCH (p1)-[r : "+rel+"]->(p2) "
    else :
        raise "Wrong dir_rel"
    code += "DELETE r"
    
    graph.run(code)

In [15]:
suppr_relation(n1,n2,rel,dir_rel)

## Afficher des informations utiles

### Qui reste-il à convaincre d'un projet ?

In [16]:
def à_convaincre(nom_du_projet):
    query = """
    MATCH (p:Person)-[rel:EMPLOYE_DANS]-(c:Company)
    WHERE p.position_"""+nom_du_projet+""" = "A convaincre"
    RETURN p.nom AS nom, c.titre AS entreprise, rel.position AS position
    """

    return graph.run(query).to_data_frame()

In [17]:
à_convaincre("D")

,nom,entreprise,position
0,Anne Gautier,DCorp,Technique
1,Joseph Thomas,DCorp,Technique


Certains détracteurs subsistent : à contacter !

### Trouver la distance entre deux personnes

#### En fonction de leurs positions et de leurs entreprises

In [36]:
def distance_pos(A,B):
    query = """
    MATCH (a:Person)-[:EMPLOYE_DANS {position:'"""+A[0]+"""'}]-(:Company {titre:'"""+A[1]+"""'})
    MATCH (b:Person)-[:EMPLOYE_DANS {position:'"""+B[0]+"""'}]-(:Company {titre:'"""+B[1]+"""'})
    MATCH p = shortestPath((a)-[*]-(b))
    RETURN a.nom,b.nom,length(p) AS distance"""
    return graph.run(query).to_data_frame()

In [37]:
A = ["CEO","ECorp"]
B = ["CFO","OBS"]

In [38]:
distance_pos(A,B)

,a.nom,b.nom,distance
0,Joséphine Michel,Lucien Martin,2


#### En fonction de leurs noms

In [39]:
def distance_noms(nomA,nomB):
    query = "MATCH p = shortestPath((a:Person {nom:'"+nomA+"'})-[*]-(b:Person {nom:'"+nomB+"'})) RETURN a.nom, b.nom, length(p) AS distance"
    return graph.run(query).to_data_frame()

In [40]:
nomA = "Joseph Leclerc"
nomB = "Auguste Petit"

In [41]:
distance_noms(nomA,nomB)

,a.nom,b.nom,distance
0,Joseph Leclerc,Auguste Petit,2


### Quels partenaires n'ont aucune relation directe avec OBS (ne sont pas "surveillés") ?

In [42]:
def a_connaitre():
    query = """
    MATCH (a:Company)-[:EMPLOYE_DANS]-(p:Person)
    WHERE NOT a.titre = "OBS"
    WITH p
    MATCH (c:Company {titre:"OBS"})
    MATCH short=shortestpath((c)-[*]-(p))
    WITH p,length(short) AS len
    WHERE len > 2
    RETURN p.nom AS nom
    """
    return graph.run(query).to_data_frame()

In [43]:
a_connaitre()

,nom
0,Marthe Guerin
1,Maurice Duval
2,Pierre Leclerc
3,Henriette Bonnet
4,Renée Morin
5,Jules Joly
6,Anne Gautier
7,Alice Fournier
8,Maria Payet
9,Georgette Gerard


Nous avons une liste de 12 personnes potentielles à contacter !

## Afficher le graphique

A faire.